In [ ]:
import pandas as pd
import numpy as np

## Load Dataset

In [ ]:
df_artworks = pd.read_csv('./collection/Artworks.csv')
#Necesary to drop repited columns
df_artists = pd.read_csv('./collection/Artists.csv')

### Explore Arttwork dataset

In [ ]:
columns = list(filter( lambda x : x not in list(df_artists.columns), list(df_artworks.columns)))
df_artworks = df_artworks[columns]

In [ ]:
df_artworks.head()

**Shape of artworks dataset**

In [ ]:
df_artworks.shape

**Columns with NaN**

In [ ]:
df_artworks.isnull().any()

**Date distribution**

**Check differents types of date**

In [ ]:
dates = df_artworks['Date'].dropna()
#drop Unknown values 
dates = dates[(dates != 'n.d.') & (dates != 'Unknown')]
dates.shape

In [ ]:
#Fast way to obtain dates (could be extracting wrong dates)
data_creation = dates.str.extract(r'(\d{4})')
data_creation = data_creation.dropna()
data_creation = pd.to_numeric(data_creation[0])
data_creation = data_creation.rename('Date')
data_creation.head()

In [ ]:
data_creation.hist(bins=100)

In [ ]:
# e.g 1987
date_dddd = dates[dates.str.match(r'\d{4}$')]
print(date_dddd.shape)
date_dddd.head()

In [ ]:
dates = dates.drop(date_dddd.index)
dates.head()

In [ ]:
#e.g 1983-...
date_period = dates[dates.str.match(r'\d{4}[- –].+')]
print(date_period.shape)
date_period.head()

In [ ]:
dates = dates.drop(date_period.index)
dates.head()

In [ ]:
#e.g. c. 1934...
date_c = dates[dates.str.match(r'^c\. \d{4}.*')]
print(date_c.shape)
date_c.head()

In [ ]:
dates = dates.drop(date_c.index)
dates.head()

In [ ]:
# e.g. 1968.
date_dddd_point = dates[dates.str.match(r'^\d{4}\.$')]
print(date_dddd_point.shape)
date_dddd_point.head()

**Classification distribution**

In [ ]:
df_artworks['Classification'].value_counts().plot.bar()

**Cataloged distribution**

In [ ]:
df_artworks['Cataloged'].value_counts()

### Artwork image extraction

In [ ]:
from bs4 import BeautifulSoup
import urllib
import os
import zipfile

In [ ]:
base_url = 'https://www.moma.org/'
base_image_path = os.path.join(os.getcwd(), 'collection/images')

**Drop artworks without link**

In [ ]:
df_artworks_download = df_artworks.dropna(subset=['URL'])
df_artworks_download.head()

In [ ]:
df_artworks_download.shape

**Download artwork's images**

In [ ]:
# w to write a zipfile || a to append to an existing file
zf = zipfile.ZipFile(os.path.join(base_image_path, 'image.zip'), 'a')
zf.write(base_image_path)

In [ ]:
def download_image(x):
    global zf
    try:
        html = urllib.request.urlopen(x['URL'])
        soup = BeautifulSoup(html)
        imgs = soup.findAll('img')
        image_url = base_url + imgs[0].get('src')
        path = os.path.join(base_image_path, str(x.name)+'.jpg')
        urllib.request.urlretrieve(image_url, path)
        zf.write(path)
        os.remove(path)
        return path
    except:
        return 'error at index ' + str(x.name)

In [ ]:
#Select a window
df_to_download = df_artworks_download.loc[50001:65000]

df_to_download['Image path'] = df_to_download.apply(download_image, axis=1)

In [ ]:
df_to_download.head()

In [ ]:
zf.close()

In [ ]:
df_to_download.to_csv('./tmp/download05.csv')

### Check data download

In [ ]:
zfr = zipfile.ZipFile(os.path.join(base_image_path, 'image.zip'), 'r')

In [ ]:
len(zfr.namelist())

In [ ]:
zfr.namelist()[-1]

In [ ]:
zfr.close()

In [ ]:
df_last_download = pd.read_csv('./tmp/download04.csv')
df_last_download.tail()

In [ ]:
df_artworks['URL'][0]

In [ ]:
html = urllib.request.urlopen(df_artworks['URL'][0])
soup = BeautifulSoup(html)

In [ ]:
artwork_name = soup.findAll('span', {'class': 'work__short-caption__text--primary balance-text'})
artwork_name[0].text

In [ ]:
imgs = soup.findAll('img')

In [ ]:
image_url = base_url + imgs[0].get('src')

In [ ]:
urllib.request.urlretrieve(image_url, os.path.join(base_image_path, '0.jpg') )

In [ ]:
urllib.urlretrieve('/media/W1siZiIsIjU5NDA1Il0sWyJwIiwiY29udmVydCIsIi1yZXNpemUgMjAwMHgyMDAwXHUwMDNlIl1d.jpg?sha=7a282c9a6f26636d', os.path.basename('example'))